<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [ ]:
#A={a1, a2, a3}
#countPos = 3

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

In [2]:
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [8]:
def x1(x, v):
  return math.sin(x[0]**2 + math.log(math.fabs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(x[2] * v[2]) - 3**(v[0]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return x[0] * v[0] * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] + 4

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [17]:
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================

def createVariants(function, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(generationRandomVector(), generationRandomVector()))
  return vector

#=================================================================
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps):
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#=================================================================
#count(x, v, y)
def createDOM(functions):
  table = []
  columns = ['x1(0)', 'x2(0)', 'x3(0)', 'v1(0)', 'v2(0)', 'v3(0)', 'y1(0)', 'y2(0)', 'y3(0)']
  for i in range(len(functions)):
    table.append(createVariants(functions[i]))

  table = pd.DataFrame(np.matrix(table).T, columns = columns)
  return table

def createColumns(steps):

  return 0

In [18]:
createDOM(functions)

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,0.198857,0.459617,0.896398,534.669606,17.393140,2.266387,1.585273,0.998167,3.731607
1,0.056230,-0.964828,-0.998505,2.974496,53.037392,12.690227,-13.437892,0.987332,5.149467
2,0.990077,0.511144,0.619044,5.105933,98.418352,0.630220,0.147376,0.905080,5.774661
3,-0.604395,-0.076964,-0.542499,7.551173,-2.171099,1.609543,0.002116,0.853466,6.879776
4,0.947116,0.118857,0.229196,3.167140,-94.073285,5.350953,-1.736663,0.871890,3.636191
...,...,...,...,...,...,...,...,...,...
95,0.941751,-0.328581,-0.965219,-9.381882,-14.791000,0.829352,1.574116,0.585407,5.127061
96,0.548191,-0.955126,0.520812,3.835892,-58.608033,-0.819877,-2.635831,0.817593,-1.795271
97,0.269570,-0.865705,-0.865262,7.362834,27.294697,0.821539,12.448742,0.711555,5.082024
98,0.563876,0.502078,-0.998435,67.998295,47.272624,1.249203,63.942137,0.998550,5.202222


In [5]:
table_t0 = pd.DataFrame()
#column = ['x1(0)', 'x2(0)', 'x3(0)', 'v1(0)', 'v2(0)', 'v3(0)']

table_t0['x1(0)'] = createVariants(x1)
table_t0['x2(0)'] = createVariants(x2)
table_t0['x3(0)'] = createVariants(x3)
table_t0['v1(0)'] = createVariants(v1)
table_t0['v2(0)'] = createVariants(v2)
table_t0['v3(0)'] = createVariants(v3)

print("Исходные данные при t = 0")
table_t0

Исходные данные при t = 0


,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0)
0,0.936205,0.697263,-0.362996,-0.045941,71.151088,0.824816
1,-0.564210,0.237056,0.209940,-0.334885,-0.239720,-2.041702
2,-0.902865,0.273081,-0.708386,-6.735271,-33.792935,1.011760
3,-0.784006,0.100848,0.233036,1572.175630,58.024160,0.110352
4,0.789723,-0.477876,0.044994,-0.668011,-3.185136,1.849754
...,...,...,...,...,...,...
95,-0.604630,-0.968650,-0.593919,0.118179,-74.342145,12.531980
96,-0.827604,-0.732743,-0.378539,9.426926,-9.502358,6.022898
97,0.576601,0.315866,0.269566,-0.281922,88.178417,2.868434
98,0.224997,0.148626,0.107142,0.180532,-82.641168,4.298792


In [6]:
table_t1 = pd.DataFrame()
#column = ['x1(0)', 'x2(0)', 'x3(0)']

table_t1['x1(1)'] = createVariants(x1)
table_t1['x2(1)'] = createVariants(x2)
table_t1['x3(1)'] = createVariants(x3)

print("t = 1")
table_t1

t = 1


,x1(1),x2(1),x3(1)
0,-0.994642,0.946078,0.383448
1,0.999827,-0.557292,-0.859503
2,0.779168,-0.957475,0.933260
3,-0.294192,0.083992,-0.993719
4,-0.335718,0.712125,0.831527
...,...,...,...
95,0.279544,0.993056,0.899628
96,0.834643,-0.937092,-0.349394
97,0.777475,-0.267033,-0.158888
98,0.988598,0.458740,0.928208
